In [1]:
!pip install numba


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
from sklearn.impute import KNNImputer
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import sys 
from typing import List, Optional, Callable, Dict, Union
from collections import defaultdict
import calendar
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed

import polars as pl
from pathlib import Path
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Union, Optional, Callable
import polars as pl
from IPython.core.debugger import set_trace 
from numba import njit
# This code was developed with the assistance of multiple AI agents (e.g., Perplexity, August 2025).


In [3]:
%pwd

'C:\\Users\\mangl\\Desktop\\capstone\\pair_selection'

In [14]:
list(Path(f"../downloaded_files/Cash Data January 2021").iterdir())[:10]

[WindowsPath('../downloaded_files/Cash Data January 2021/.CNX100.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/.CNXIT.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/.NSEBANK.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/.NSEI.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/20MICRONS.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/21STCENMGM.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/3IINFOTECH.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/3MINDIA.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/3PLAND.csv'),
 WindowsPath('../downloaded_files/Cash Data January 2021/5PAISA.csv')]

In [6]:
pd.read_csv(DATA_DIR/'21STCENMGM.csv')

,<ticker>,<date>,<time>,<open>,<high>,<low>,<close>,<volume>,<o/i>
0,21STCENMGM,01/01/2021,09:45:00,10.65,10.65,10.65,10.65,11,0
1,21STCENMGM,01/01/2021,09:57:00,10.65,10.65,10.65,10.65,35,0
2,21STCENMGM,01/01/2021,09:59:00,10.65,10.65,10.65,10.65,90,0
3,21STCENMGM,01/01/2021,10:10:00,10.65,10.65,10.65,10.65,10,0
4,21STCENMGM,01/01/2021,10:13:00,10.65,10.65,10.65,10.65,139,0
...,...,...,...,...,...,...,...,...,...
521,21STCENMGM,01/29/2021,13:27:00,11.10,11.10,11.10,11.10,299,0
522,21STCENMGM,01/29/2021,14:47:00,11.15,11.15,11.15,11.15,3,0
523,21STCENMGM,01/29/2021,15:26:00,11.15,11.15,11.15,11.15,20,0
524,21STCENMGM,01/29/2021,15:28:00,11.15,11.15,11.15,11.15,8,0


In [7]:
tickers = [i.rsplit(".", maxsplit=1)[0] for i in os.listdir(DATA_DIR)]
tickers[:10] # few available tickers

['.CNX100',
 '.CNXIT',
 '.NSEBANK',
 '.NSEI',
 '20MICRONS',
 '21STCENMGM',
 '3IINFOTECH',
 '3MINDIA',
 '3PLAND',
 '5PAISA']

In [8]:
 
def get_stock_data(
    tickers: List[str],
    start: Optional[Union[str, datetime]] = None,
    end: Optional[Union[str, datetime]] = None,
    agg_func: Optional[Union[str, Callable]] = None,
    resample_freq: Optional[str] = None,
    select_columns: Optional[List[str]] = None,
    impute: bool = True,
    DATA_DIR: Optional[Union[str, Path]] = None
) -> Dict[str, pl.DataFrame]:
    """
    Load and process intraday stock data from CSV files using Polars.

    Parameters
    ----------
    tickers : List[str]
        List of stock ticker symbols (without file extensions).
    start : str or datetime, optional
        Start datetime (format: "YYYY-MM-DD" or datetime object).
    end : str or datetime, optional
        End datetime (format: "YYYY-MM-DD" or datetime object).
    agg_func : str or callable, optional
        Aggregation function for resampling (e.g., "mean", "sum").
    resample_freq : str, optional
        Resample frequency (e.g., "5m", "15m").
    columns : List[str], optional
        List of columns to retain (e.g., ["open_price", "close_price"]).
    impute : bool, default True
        Whether to impute missing data using forward fill.
    DATA_DIR : str or Path, optional
        Directory containing the CSV files.

    Returns
    -------
    Dict[str, pl.DataFrame]
        Dictionary mapping each ticker to its processed DataFrame.
    """
    # Convert DATA_DIR to Path
    DATA_DIR = Path(DATA_DIR)

    rev_cols_map = {
        "ticker": "<ticker>",
        "date": "<date>",
        "time": "<time>",
        "open_price": "<open>",
        "high_price": "<high>",
        "low_price": "<low>",
        "close_price": "<close>",
        "volume": "<volume>",
        "open_interest": "<o/i> "
    }
    new_columns, columns = zip(*rev_cols_map.items())
    if select_columns:    
        new_columns = select_columns + ["date", "time"]
        columns = [rev_cols_map[i] for i in new_columns]

    data_dict = {}
    
    start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")  
    end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S")  
    
    for ticker in tickers:
        # Read CSV
        df = pl.read_csv(DATA_DIR / f"{ticker}.csv", columns=columns).rename(self._cols_map)

        # Create datetime column
        df = df.with_columns(
            pl.concat_str([df["date"], df["time"]], separator=" ")
            .str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M:%S")
            .alias("datetime")
        ).sort("datetime")

        # Drop extra columns
        df = df.drop(["ticker", "date", "time"])
        df = df.set_sorted("datetime")

        # Impute missing values
        if impute:
            df = df.upsample(
                time_column="datetime",
                every="1m",
                maintain_order=True
            ).fill_null(strategy="forward")
        df = df.filter((pl.col("datetime") >= start) & (pl.col("datetime") <= end))

        # Resample if requested
        if resample_freq:
            if not agg_func:
                agg_func = "mean"
            df = df.group_by_dynamic(
                time_column="datetime",
                every=resample_freq,
                closed="left"
            ).agg({col: agg_func for col in df.columns if col != "datetime"})

        

        # Keep only requested columns + datetime
        keep_cols = ["datetime"] + [col for col in new_columns if col in df.columns]
        df = df.select(keep_cols)

        data_dict[ticker] = df

    return data_dict

In [16]:
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Optional, Union, Dict, Callable
import polars as pl
import calendar


class StockDataLoader:
    def __init__(
        self,
        tickers: List[str],
        start: Union[str, datetime],
        end: Union[str, datetime],
        agg_func: Optional[Union[str, Callable]] = None,
        resample_freq: Optional[str] = None,
        select_columns: Optional[List[str]] = None,
        impute: bool = True,
        base_dir: Optional[Union[str, Path]] = "../downloaded_files",
    ):
        # Make attributes private
        self._start = self._parse_datetime(start)
        self._end = self._parse_datetime(end)
        self._base_dir = Path(base_dir)
        
        self._tickers = tickers or self._find_available_tickers()
        
        self._agg_func = agg_func
        self._resample_freq = resample_freq
        self._select_columns = select_columns  
        self._impute = impute

        # Cache for storing loaded data
        self._data_cache: Dict[str, pl.DataFrame] = {}

        # Column mapping for renaming
        self._rev_cols_map = {
            "open_interest": "<o/i> ",
            "date": "<date>",
            "time": "<time>",
            "open": "<open>",
            "high": "<high>",
            "low": "<low>",
            "close": "<close>",
            "volume": "<volume>",
        }
        self._cols_map = {
            "<o/i> ": "open_interest",
            "<date>": "date",
            "<time>": "time",
            "<open>": "open",
            "<high>": "high",
            "<low>": "low",
            "<close>": "close",
            "<volume>": "volume"
        }
        self._new_columns, self._columns = zip(*self._rev_cols_map.items())
        if self._select_columns:
            self._new_columns = self._select_columns + ["date", "time"]
            self._columns = [self._rev_cols_map[c] for c in self._new_columns]

        self._schema = {
             "date": pl.String,
            "time":pl.String,
            "datetime": pl.Datetime("us"),
            "open_price": pl.Float64,
            "high_price": pl.Float64,
            "low_price": pl.Float64,
            "close_price": pl.Float64,
            "volume": pl.Float64,
            "open_interest": pl.Float64,
        }
    def _find_available_tickers(self) -> List[str]:
        """Find all tickers present across all months by set intersection."""
        months = self._generate_monthly_files()
        all_month_tickers = []

        for month in months:
            folder = self._get_month_dir(month)
            if not folder.exists():
                continue

            tickers_in_month = {
                f.stem for f in folder.glob("*.csv") if f.is_file()
            }
            if tickers_in_month:
                all_month_tickers.append(tickers_in_month)

        # If no CSVs found at all
        if not all_month_tickers:
            raise FileNotFoundError(
                f"No CSV files found in any month between {self._start} and {self._end}"
            )

        # Take intersection to get tickers present in *all* months
        common_tickers = set.intersection(*all_month_tickers)
        if not common_tickers:
            raise ValueError(
                "No common tickers found across all months in the given date range."
            )

        return sorted(common_tickers)
    def _parse_datetime(self, dt):
        """Ensure datetime parsing."""
        if isinstance(dt, datetime):
            return dt
        return datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")

    def _generate_monthly_files(self):
        """Generate list of months between start and end."""
        start_month = self._start.replace(day=1)
        end_month = self._end.replace(day=1)

        months = []
        while start_month <= end_month:
            months.append(start_month.strftime("%Y-%m"))  # e.g. "2025-01"
            start_month += timedelta(days=32)
            start_month = start_month.replace(day=1)
        return months

    def _get_month_dir(self, month: str) -> Path:
        """
        Construct the directory path dynamically.
        Example: "../downloaded_files/Cash Data January 2025"
        """
        year, month_num = month.split("-")
        month_name = calendar.month_name[int(month_num)]
        return self._base_dir / f"Cash Data {month_name} {year}"

    def _empty_df(self) -> pl.DataFrame:
        """Return an empty DataFrame with correct schema."""
        return pl.DataFrame(schema=self._schema)

    def _load_single_file(self, ticker: str, month: str) -> pl.DataFrame:
        """Load and preprocess a single month's CSV for one ticker."""
        folder = self._get_month_dir(month)
        file_path = folder / f"{ticker}.csv"
        

        # If file doesn't exist → return empty DataFrame with correct schema
        if not file_path.exists():
            return self._empty_df()

        df = pl.read_csv(file_path, columns=self._columns)
        df = df.rename({i:self._cols_map[i] for i in df.columns})
        

        # Create datetime column
        df = df.with_columns(
            pl.concat_str([df["date"], df["time"]], separator=" ")
            .str.strptime(pl.Datetime, format="%m/%d/%Y %H:%M:%S")
            .alias("datetime")
        ).sort("datetime")

        df = df.drop([ "date", "time"])
        df = df.set_sorted("datetime")

        # Impute missing values if needed
        if self._impute:
            df = df.upsample(
                time_column="datetime",
                every="1m",
                maintain_order=True
            ).fill_null(strategy="forward")

        # Filter by start & end for partial month reads
        return df.filter((pl.col("datetime") >= self._start) & (pl.col("datetime") <= self._end))

    def _merge_monthly_data(self, ticker: str) -> pl.DataFrame:
        """Read multiple monthly files and combine them."""
        months = self._generate_monthly_files()
        dfs = [self._load_single_file(ticker, month) for month in months]
        # Always concatenate with schema alignment
        df = pl.concat(dfs, how="vertical") if dfs else self._empty_df()

        # Resample if requested
        if not df.is_empty() and self._resample_freq:
            func = self._agg_func or "mean"
            df = df.group_by_dynamic(
                time_column="datetime",
                every=self._resample_freq,
                closed="left"
            ).agg({col: func for col in df.columns if col != "datetime"})

        # Keep only required columns
        keep_cols = ["datetime"] + [col for col in self._new_columns if col in df.columns]
        return df.select(keep_cols)
     
    def get_data_for_tickers(self, tickers: Optional[List[str]] = None) -> Dict[str, pl.DataFrame]:
        """
        Public method: Returns processed data for one or more tickers.
        Uses cache when available.
        Fully parallelized using joblib with all available CPU cores.
        """
        if not tickers:
            tickers = self._tickers
        else:
            tickers = [t for t in tickers if t in self._tickers]

        if not tickers:
            raise ValueError("No valid tickers provided or found.")

        # Use all available cores
        n_jobs = os.cpu_count() or 1

        # Run in parallel for all tickers
        results_list = Parallel(n_jobs=n_jobs, backend="loky")(
            delayed(self._load_or_get_from_cache)(ticker) for ticker in tickers
        )

        # Combine ticker names with their data
        return dict(zip(tickers, results_list))

    def _load_or_get_from_cache(self, ticker: str) -> pl.DataFrame:
        """Helper to either fetch from cache or load fresh."""
        if ticker not in self._data_cache:
            self._data_cache[ticker] = self._merge_monthly_data(ticker)
        return self._data_cache[ticker]

In [17]:
loader = StockDataLoader(
    tickers=[],
    start="2021-12-02 09:15:00",
    end="2022-03-02 15:30:00",
    # resample_freq="5m",
    select_columns=["volume"],
    base_dir="../downloaded_files"
)


In [18]:
data_dict = loader.get_data_for_tickers();

In [12]:
# data_dict = get_stock_data(
#     tickers=tickers,
#     start="2021-01-02 9:15:00",
#     end="2021-01-05 10:40:00",
#     agg_func="mean",
#     # resample_freq="5m",
#     # impute=True,
#     DATA_DIR=DATA_DIR
# )
# # data_dict



In [19]:
data_dict["ADORWELD"]

datetime,volume
datetime[μs],i64
2021-12-02 09:15:00,68
2021-12-02 09:16:00,68
2021-12-02 09:17:00,1
2021-12-02 09:18:00,1
2021-12-02 09:19:00,1
…,…
2022-03-02 15:26:00,6
2022-03-02 15:27:00,1
2022-03-02 15:28:00,32


In [ ]:

import ipdb
class StockDataReader:
    COLS_MAP = {
        "<ticker>": "ticker",
        "<date>": "date",
        "<time>": "time",
        "<open>": "open_price",
        "<high>": "high_price",
        "<low>": "low_price",
        "<close>": "close_price",
        "<volume>": "volume",
        "<o/i>": "open_interest"
    }

    REV_COLS_MAP = {v: k for k, v in COLS_MAP.items()}

    def __init__(self, data_dir: Path):
        self.data_dir = Path(data_dir)

    def _get_file_paths(self, ticker: str, start: pd.Timestamp, end: pd.Timestamp):
        """
        Generate all possible file paths for a ticker between start and end dates.
        Expects folder names like "Cash Data January 2021".
        """
        file_paths = []
        
        # Normalize start and end to the first of their months
        start = start.normalize().replace(day=1)
        end = end.normalize().replace(day=1)
        
        # Iterate over each month start in range
        for ts in pd.date_range(start, end, freq="MS"):
            month_name = calendar.month_name[ts.month]
            year = ts.year
            folder = self.data_dir / f"Cash Data {month_name} {year}"
            
            file_path = folder / f"{ticker}.csv"
            if file_path.exists():
                file_paths.append(file_path)
        
        return file_paths


    def _read_single_ticker(
        self, ticker: str, start=None, end=None, agg_func=None,
        resample_freq=None, columns=None, impute=True
    ) -> pd.DataFrame:
        """Load and process a single ticker CSV into a cleaned DataFrame"""
        start = pd.to_datetime(start) 
        end = pd.to_datetime(end)  
        file_paths = self._get_file_paths(ticker, start, end)
        
        dfs = []
        for file_path in file_paths:
            df = pd.read_csv(file_path)
            df = df.rename(columns=self.COLS_MAP)
            dfs.append(df)

        df = pd.concat(dfs, ignore_index=True)

        # Create datetime column
        df["datetime"] = pd.to_datetime(
            df["date"] + " " + df["time"], format="%m/%d/%Y %H:%M:%S"
        )
        df = df.drop(columns=["ticker", "date", "time"], errors="ignore")
        df = df.set_index("datetime").sort_index()

        if resample_freq:
            impute = True

        # Impute missing data
        if impute and not df.empty:
            full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="1min")
            df = df.reindex(full_index)
            for col in df.columns:
                df[col] = df[col].fillna(df[col].expanding().median())


        df = df[(df.index >= start) & (df.index <= end)]

        # Resample if requested
        if resample_freq:
            if agg_func is None:
                raise ValueError("agg_func must be provided when resampling.")
            df = df.resample(resample_freq).agg(agg_func)

        return df

    def get_stock_data(
        self, tickers, start=None, end=None, agg_func=None,
        resample_freq=None, columns=None, impute=True
    ) -> dict: 
        """Load and process multiple tickers into a dict of DataFrames"""
        data_dict = {}
        for ticker in tickers:
            df = self._read_single_ticker(
                ticker,
                start=start,
                end=end,
                agg_func=agg_func,
                resample_freq=resample_freq,
                columns=columns,
                impute=impute,
            )
            data_dict[ticker] = df
        return data_dict

    def get_stock_data(
        self, tickers, start=None, end=None, agg_func=None,
        resample_freq=None, columns=None, impute=True, n_jobs=-1
    ) -> dict:
        """Load and process multiple tickers into a dict of DataFrames (parallelized)."""

        def process_one(ticker):
            try:
                df = self._read_single_ticker(
                    ticker,
                    start=start,
                    end=end,
                    agg_func=agg_func,
                    resample_freq=resample_freq,
                    columns=columns,
                    impute=impute,
                )
                return ticker, df
            except Exception as e:
                # you can log/print here if debugging
                return ticker, None

        results = Parallel(n_jobs=n_jobs)(
            delayed(process_one)(ticker) for ticker in tickers
        )

        # collect only valid DataFrames
        data_dict = {ticker: df for ticker, df in results if df is not None}
        return data_dict

In [ ]:
# reader = StockDataReader(f"../downloaded_files")

# # Read 1-minute data for AAPL, impute missing minutes
# df_dict = reader.get_stock_data(
#     ["RELIANCE", "3MINDIA"], 
#     start="2021-10-05 09:15", 
#     end="2022-01-05 15:30", 
#     columns=["date", "time", "close", "volume"]
# )



In [ ]:
import os
import calendar
from pathlib import Path
from collections import defaultdict
from joblib import Parallel, delayed
import pandas as pd
import pickle

ILLIQUID_TICKERS = set()
PICKLE_FILE = Path("illiquid_tickers.pkl")


def load_illiquid_set():
    global ILLIQUID_TICKERS
    if PICKLE_FILE.exists():
        with open(PICKLE_FILE, "rb") as f:
            ILLIQUID_TICKERS = pickle.load(f)
    else:
        ILLIQUID_TICKERS = set()


def save_illiquid_set():
    with open(PICKLE_FILE, "wb") as f:
        pickle.dump(ILLIQUID_TICKERS, f)


def compute_liquidity_metrics(df: pd.DataFrame, ticker: str, lookback_days: int = 20,
                              vol_threshold: int = 1000, zero_ratio_threshold: float = 0.15,
                              amihud_threshold: float = 0.0001) -> pd.DataFrame:
    """
    Compute liquidity metrics and return a NEW DataFrame with prefixed columns.
    """
    metrics_df = df.copy().sort_index()

    # Rolling and exponential volume
    metrics_df['liquid_rolling_vol'] = metrics_df['volume'].rolling(lookback_days).mean()
    metrics_df['liquid_ewm_vol'] = metrics_df['volume'].ewm(span=lookback_days, adjust=False).mean()

    # Zero-volume ratio
    metrics_df['liquid_zero_vol_ratio'] = (metrics_df['volume'] == 0).rolling(lookback_days).mean()

    # Amihud Illiquidity
    if {'close_price', 'open_price'}.issubset(metrics_df.columns):
        metrics_df['return'] = metrics_df['close_price'].pct_change()
        metrics_df['liquid_amihud'] = (metrics_df['return'].abs() / metrics_df['volume']).rolling(lookback_days).mean()

    # ---- Determine liquidity flag ----
    last_row = metrics_df.iloc[-1]
    is_liquid = True

    if last_row['liquid_rolling_vol'] < vol_threshold:
        is_liquid = False
    if last_row['liquid_zero_vol_ratio'] > zero_ratio_threshold:
        is_liquid = False
    if 'liquid_amihud' in metrics_df.columns and last_row['liquid_amihud'] > amihud_threshold:
        is_liquid = False

    metrics_df['liquid_is_liquid'] = is_liquid

    return metrics_df, is_liquid
def process_trade_day(trade_day, reader: StockDataReader,
                      vol_threshold=1000, zero_ratio_threshold=0.15,
                      amihud_threshold=0.0001, lookback_days=20):
    """
    Process all tickers for a given trade day, filter illiquid stocks,
    and return dict of liquid DataFrames.
    """
    global ILLIQUID_TICKERS
    load_illiquid_set()  # load saved set first

    trade_day = pd.Timestamp(trade_day)

    # Build month folder
    month_name = calendar.month_name[trade_day.month]
    year = trade_day.year
    DATA_DIR = reader.data_dir / f"Cash Data {month_name} {year}"
    if not DATA_DIR.exists():
        return {}

    # Collect tickers for that folder
    tickers = [f.rsplit(".", 1)[0] for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
    tickers = list(set(tickers) - ILLIQUID_TICKERS)

    # Batch load data (parallelized inside reader)
    dfs = reader.get_stock_data(
        tickers,
        start=trade_day.replace(hour=9, minute=15),
        end=trade_day.replace(hour=15, minute=30),
        columns=["date", "time", "close", "volume", "open"],
        impute=False
    )

    liquid_dfs = {}
    for ticker, df in dfs.items():
        try:
            metrics_df, is_liquid = compute_liquidity_metrics(
                df, ticker,
                lookback_days=lookback_days,
                vol_threshold=vol_threshold,
                zero_ratio_threshold=zero_ratio_threshold,
                amihud_threshold=amihud_threshold
            )

            if is_liquid:
                liquid_dfs[ticker] = metrics_df
            else:
                ILLIQUID_TICKERS.add(ticker)

        except Exception:
            # skip bad ticker
            continue

    save_illiquid_set()
    return liquid_dfs


reader = StockDataReader("../downloaded_files")

liquid_data = process_trade_day(
    "2021-10-05", 
    reader,
    vol_threshold=1000,
    zero_ratio_threshold=0.15
)

print("Liquid tickers:", list(liquid_data.keys())[:10])
print("Illiquid tickers:", list(ILLIQUID_TICKERS)[:10])


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import defaultdict
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


class AdaptiveLiquidityPredictor:
    def __init__(self, data_reader, lookback_days=20, normalization_method='robust'):
        self.reader = data_reader
        self.lookback_days = lookback_days
        self.normalization_method = normalization_method
        
        # EMA parameters for different time horizons
        self.ema_params = {
            'fast': 0.3,      # For recent 15-min updates
            'medium': 0.1,    # For hourly patterns  
            'slow': 0.05      # For daily/historical patterns
        }
        
        # Liquidity state tracking
        self.liquidity_states = defaultdict(dict)
        self.historical_features = defaultdict(list)
        
        # Normalization components
        self.setup_normalizers()
        
        # ML Models
        self.volume_predictor = None
        self.amihud_predictor = None
        
        # Model storage
        self.model_path = Path("liquidity_models.pkl")
        self.normalization_path = Path("normalization_params.pkl")
        self.load_models()
        self.load_normalization_params()

    def setup_normalizers(self):
        """Initialize different types of normalizers for different data types"""
        if self.normalization_method == 'robust':
            # Robust to outliers - good for financial data
            self.feature_scaler = RobustScaler()
            self.volume_scaler = RobustScaler()
            self.amihud_scaler = RobustScaler()
        elif self.normalization_method == 'standard':
            self.feature_scaler = StandardScaler()
            self.volume_scaler = StandardScaler()
            self.amihud_scaler = StandardScaler()
        else:  # minmax
            self.feature_scaler = MinMaxScaler()
            self.volume_scaler = MinMaxScaler()
            self.amihud_scaler = MinMaxScaler()
        
        # Separate scalers for different metric types
        self.price_scaler = RobustScaler()  # For price-based metrics
        self.ratio_scaler = MinMaxScaler()   # For ratios (already bounded)
        
        # Normalization parameters for EMA updates
        self.normalization_params = {
            'volume_stats': {'mean': 0, 'std': 1, 'q25': 0, 'q75': 1},
            'amihud_stats': {'mean': 0, 'std': 1, 'q25': 0, 'q75': 1},
            'price_stats': {'mean': 0, 'std': 1, 'q25': 0, 'q75': 1},
            'initialized': False
        }

    def update_normalization_params(self, training_data):
        """Update normalization parameters from training data"""
        volumes, amihuds, prices = [], [], []
        
        for ticker, ticker_data in training_data.items():
            if isinstance(ticker_data, pd.DataFrame):
                if 'volume' in ticker_data.columns:
                    volumes.extend(ticker_data['volume'].dropna().values)
                if 'close' in ticker_data.columns:
                    prices.extend(ticker_data['close'].dropna().values)
                    # Calculate returns for Amihud
                    returns = ticker_data['close'].pct_change().dropna()
                    volume_safe = ticker_data['volume'].replace(0, 1e-8)
                    amihud_values = (returns.abs() / volume_safe).dropna().values
                    amihuds.extend(amihud_values[amihud_values < np.inf])
        
        if volumes:
            volumes = np.array(volumes)
            self.normalization_params['volume_stats'] = {
                'mean': np.mean(volumes),
                'std': np.std(volumes) if np.std(volumes) != 0 else 1,
                'q25': np.percentile(volumes, 25),
                'q75': np.percentile(volumes, 75)
            }
        
        if amihuds:
            amihuds = np.array(amihuds)
            # Handle extreme values
            amihuds = amihuds[amihuds < np.percentile(amihuds, 95)]
            self.normalization_params['amihud_stats'] = {
                'mean': np.mean(amihuds),
                'std': np.std(amihuds) if np.std(amihuds) != 0 else 1,
                'q25': np.percentile(amihuds, 25),
                'q75': np.percentile(amihuds, 75)
            }
        
        if prices:
            prices = np.array(prices)
            self.normalization_params['price_stats'] = {
                'mean': np.mean(prices),
                'std': np.std(prices) if np.std(prices) != 0 else 1,
                'q25': np.percentile(prices, 25),
                'q75': np.percentile(prices, 75)
            }
        
        self.normalization_params['initialized'] = True
        self.save_normalization_params()

    def normalize_metric(self, value, metric_type, method='robust'):
        """Normalize individual metrics using stored parameters"""
        if not self.normalization_params['initialized']:
            return value
        
        stats = self.normalization_params.get(f'{metric_type}_stats', {})
        
        if method == 'robust':
            # Robust normalization using quartiles
            iqr = stats.get('q75', 1) - stats.get('q25', 0)
            if iqr == 0:
                iqr = 1
            return (value - stats.get('q25', 0)) / iqr
        elif method == 'standard':
            # Z-score normalization
            std = stats.get('std', 1)
            if std == 0:
                std = 1
            return (value - stats.get('mean', 0)) / std
        else:
            return value

    def denormalize_metric(self, normalized_value, metric_type, method='robust'):
        """Denormalize predictions back to original scale"""
        if not self.normalization_params['initialized']:
            return normalized_value
        
        stats = self.normalization_params.get(f'{metric_type}_stats', {})
        
        if method == 'robust':
            iqr = stats.get('q75', 1) - stats.get('q25', 0)
            if iqr == 0:
                iqr = 1
            return normalized_value * iqr + stats.get('q25', 0)
        elif method == 'standard':
            std = stats.get('std', 1)
            if std == 0:
                std = 1
            return normalized_value * std + stats.get('mean', 0)
        else:
            return normalized_value

    def extract_time_features(self, timestamp):
        """Extract time-based features for ML models"""
        return {
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'day_of_week': timestamp.weekday(),
            'minutes_from_open': max(0, (timestamp.hour - 9) * 60 + (timestamp.minute - 15)),
            'minutes_to_close': max(0, (15 - timestamp.hour) * 60 + (30 - timestamp.minute)),
            'is_opening': 1 if timestamp.hour == 9 and timestamp.minute <= 30 else 0,
            'is_closing': 1 if timestamp.hour >= 15 else 0,
            'is_lunch_time': 1 if 12 <= timestamp.hour <= 13 else 0
        }

    def compute_base_liquidity_metrics(self, df, ticker):
        """Compute base liquidity metrics from historical data with proper normalization"""
        if df is None or df.empty:
            return None
        
        df_sorted = df.sort_index()
        
        # Volume metrics with outlier handling
        volume_clean = df_sorted['volume'].clip(lower=0, upper=df_sorted['volume'].quantile(0.99))
        rolling_vol = volume_clean.rolling(window=f'{self.lookback_days}D', min_periods=1).mean()
        ewm_vol = volume_clean.ewm(span=self.lookback_days, adjust=False).mean()
        vol_std = volume_clean.rolling(window=f'{self.lookback_days}D', min_periods=1).std()
        
        # Volume normalization
        vol_median = volume_clean.rolling(window=f'{self.lookback_days}D', min_periods=1).median()
        vol_mad = (volume_clean - vol_median).abs().rolling(window=f'{self.lookback_days}D', min_periods=1).median()
        
        # Zero volume ratio
        zero_vol_ratio = (df_sorted['volume'] == 0).rolling(window=f'{self.lookback_days}D', min_periods=1).mean()
        
        # Price-based metrics with normalization
        if 'close' in df_sorted.columns:
            # Clean price data
            prices = df_sorted['close'].fillna(method='ffill')
            returns = prices.pct_change().fillna(0)
            
            # Winsorize extreme returns
            return_q99 = returns.quantile(0.99)
            return_q01 = returns.quantile(0.01)
            returns_clean = returns.clip(lower=return_q01, upper=return_q99)
            
            # Amihud illiquidity with safe division
            volume_safe = volume_clean.replace(0, volume_clean.median() * 0.01 or 1e-8)
            amihud_raw = (returns_clean.abs() / volume_safe)
            
            # Robust Amihud calculation
            amihud = amihud_raw.rolling(window=f'{self.lookback_days}D', min_periods=1).median()
            amihud_std = amihud_raw.rolling(window=f'{self.lookback_days}D', min_periods=1).std()
        else:
            amihud = pd.Series(0, index=df_sorted.index)
            amihud_std = pd.Series(0, index=df_sorted.index)
            returns_clean = pd.Series(0, index=df_sorted.index)
        
        # Volatility with normalization
        volatility = returns_clean.rolling(window=f'{self.lookback_days}D', min_periods=1).std()
        
        # Get latest values with safe indexing
        metrics = {
            'rolling_vol': rolling_vol.iloc[-1] if not rolling_vol.empty else 0,
            'ewm_vol': ewm_vol.iloc[-1] if not ewm_vol.empty else 0,
            'vol_std': vol_std.iloc[-1] if not vol_std.empty else 0,
            'vol_median': vol_median.iloc[-1] if not vol_median.empty else 0,
            'vol_mad': vol_mad.iloc[-1] if not vol_mad.empty else 1,
            'zero_vol_ratio': zero_vol_ratio.iloc[-1] if not zero_vol_ratio.empty else 0,
            'amihud': amihud.iloc[-1] if not amihud.empty else 0,
            'amihud_std': amihud_std.iloc[-1] if not amihud_std.empty else 0,
            'volatility': volatility.iloc[-1] if not volatility.empty else 0
        }
        
        # Apply normalization if parameters are available
        if self.normalization_params['initialized']:
            metrics['rolling_vol_norm'] = self.normalize_metric(metrics['rolling_vol'], 'volume')
            metrics['amihud_norm'] = self.normalize_metric(metrics['amihud'], 'amihud')
        else:
            metrics['rolling_vol_norm'] = metrics['rolling_vol']
            metrics['amihud_norm'] = metrics['amihud']
            
        return metrics

    def update_ema_state(self, ticker, new_metrics, timestamp):
        """Update EMA-based liquidity state with normalized values"""
        vol_metric = new_metrics.get('rolling_vol_norm', new_metrics['rolling_vol'])
        amihud_metric = new_metrics.get('amihud_norm', new_metrics['amihud'])
        
        if ticker not in self.liquidity_states:
            self.liquidity_states[ticker] = {
                'fast_vol_ema': vol_metric,
                'medium_vol_ema': vol_metric, 
                'slow_vol_ema': vol_metric,
                'fast_amihud_ema': amihud_metric,
                'medium_amihud_ema': amihud_metric,
                'slow_amihud_ema': amihud_metric,
                'last_update': timestamp,
                'raw_vol': new_metrics['rolling_vol'],
                'raw_amihud': new_metrics['amihud']
            }
        else:
            state = self.liquidity_states[ticker]
            
            # Update volume EMAs with normalized values
            state['fast_vol_ema'] = (self.ema_params['fast'] * vol_metric + 
                                    (1 - self.ema_params['fast']) * state['fast_vol_ema'])
            state['medium_vol_ema'] = (self.ema_params['medium'] * vol_metric + 
                                     (1 - self.ema_params['medium']) * state['medium_vol_ema'])
            state['slow_vol_ema'] = (self.ema_params['slow'] * vol_metric + 
                                   (1 - self.ema_params['slow']) * state['slow_vol_ema'])
            
            # Update Amihud EMAs with normalized values
            state['fast_amihud_ema'] = (self.ema_params['fast'] * amihud_metric + 
                                      (1 - self.ema_params['fast']) * state['fast_amihud_ema'])
            state['medium_amihud_ema'] = (self.ema_params['medium'] * amihud_metric + 
                                        (1 - self.ema_params['medium']) * state['medium_amihud_ema'])
            state['slow_amihud_ema'] = (self.ema_params['slow'] * amihud_metric + 
                                      (1 - self.ema_params['slow']) * state['slow_amihud_ema'])
            
            state['last_update'] = timestamp
            state['raw_vol'] = new_metrics['rolling_vol']
            state['raw_amihud'] = new_metrics['amihud']

    def prepare_ml_features(self, ticker, timestamp, current_metrics):
        """Prepare normalized features for ML prediction"""
        time_features = self.extract_time_features(timestamp)
        
        features = {
            **time_features,
            'current_vol_norm': current_metrics.get('rolling_vol_norm', current_metrics['rolling_vol']),
            'current_amihud_norm': current_metrics.get('amihud_norm', current_metrics['amihud']),
            'vol_std_norm': current_metrics['vol_std'] / (current_metrics.get('vol_mad', 1) + 1e-8),
            'volatility_norm': current_metrics['volatility'],
            'zero_vol_ratio': current_metrics['zero_vol_ratio']
        }
        
        if ticker in self.liquidity_states:
            state = self.liquidity_states[ticker]
            features.update({
                'fast_vol_ema': state['fast_vol_ema'],
                'medium_vol_ema': state['medium_vol_ema'],
                'slow_vol_ema': state['slow_vol_ema'],
                'fast_amihud_ema': state['fast_amihud_ema'],
                'medium_amihud_ema': state['medium_amihud_ema'],
                'slow_amihud_ema': state['slow_amihud_ema']
            })
        else:
            features.update({
                'fast_vol_ema': features['current_vol_norm'],
                'medium_vol_ema': features['current_vol_norm'],
                'slow_vol_ema': features['current_vol_norm'],
                'fast_amihud_ema': features['current_amihud_norm'],
                'medium_amihud_ema': features['current_amihud_norm'],
                'slow_amihud_ema': features['current_amihud_norm']
            })
        
        if ticker in self.liquidity_states:
            state = self.liquidity_states[ticker]
            vol_momentum = features['current_vol_norm'] - state['medium_vol_ema']
            amihud_momentum = features['current_amihud_norm'] - state['medium_amihud_ema']
            
            features.update({
                'vol_momentum': vol_momentum,
                'amihud_momentum': amihud_momentum,
                'vol_ema_spread': state['fast_vol_ema'] - state['slow_vol_ema'],
                'amihud_ema_spread': state['fast_amihud_ema'] - state['slow_amihud_ema']
            })
        
        return features

    def train_models(self, training_data):
        """Train ML models on historical data with proper normalization"""
        if not training_data:
            print("No training data available")
            return
        
        print("Updating normalization parameters...")
        self.update_normalization_params(training_data)
        
        X, y_vol, y_amihud = [], [], []
        
        for ticker, ticker_data in training_data.items():
            if not isinstance(ticker_data, pd.DataFrame) or ticker_data.empty:
                continue
            metrics = self.compute_base_liquidity_metrics(ticker_data, ticker)
            if metrics is None:
                continue
            for idx, row in ticker_data.iterrows():
                timestamp = idx
                features = self.prepare_ml_features(ticker, timestamp, metrics)
                X.append(list(features.values()))
                y_vol.append(self.normalize_metric(row.get('volume', 0), 'volume'))
                y_amihud.append(self.normalize_metric(
                    (row['close'].pct_change().abs() / row['volume'] if row['volume'] != 0 else 0), 'amihud'
                ))
        
        if not X:
            print("No valid training data after processing")
            return
        
        X = np.array(X)
        y_vol = np.array(y_vol)
        y_amihud = np.array(y_amihud)
        
        X_scaled = self.feature_scaler.fit_transform(X)
        X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=1.0, neginf=-1.0)
        y_vol = np.nan_to_num(y_vol, nan=0.0, posinf=1.0, neginf=-1.0)
        y_amihud = np.nan_to_num(y_amihud, nan=0.0, posinf=1.0, neginf=-1.0)
        
        self.volume_predictor = GradientBoostingRegressor(
            n_estimators=100, 
            learning_rate=0.1,
            max_depth=6,
            min_samples_split=20,
            min_samples_leaf=10,
            random_state=42
        )
        
        self.amihud_predictor = GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1, 
            max_depth=6,
            min_samples_split=20,
            min_samples_leaf=10,
            random_state=42
        )
        
        self.volume_predictor.fit(X_scaled, y_vol)
        self.amihud_predictor.fit(X_scaled, y_amihud)
        
        vol_pred = self.volume_predictor.predict(X_scaled)
        amihud_pred = self.amihud_predictor.predict(X_scaled)
        
        vol_rmse = np.sqrt(mean_squared_error(y_vol, vol_pred))
        amihud_rmse = np.sqrt(mean_squared_error(y_amihud, amihud_pred))
        
        print(f"Models trained successfully")
        print(f"Volume RMSE: {vol_rmse:.4f}")
        print(f"Amihud RMSE: {amihud_rmse:.4f}")
        
        self.save_models()
        self.save_normalization_params()

    def predict_future_liquidity(self, ticker, timestamp, current_metrics, horizon_minutes=60):
        """Predict future liquidity using ML models"""
        if self.volume_predictor is None or self.amihud_predictor is None:
            return current_metrics['rolling_vol'], current_metrics['amihud']
        
        features = self.prepare_ml_features(ticker, timestamp, current_metrics)
        feature_array = np.array([list(features.values())])
        
        try:
            feature_array_scaled = self.feature_scaler.transform(feature_array)
            predicted_vol = self.volume_predictor.predict(feature_array_scaled)[0]
            predicted_amihud = self.amihud_predictor.predict(feature_array_scaled)[0]
            
            predicted_vol = self.denormalize_metric(predicted_vol, 'volume')
            predicted_amihud = self.denormalize_metric(predicted_amihud, 'amihud')
            
            return max(0, predicted_vol), max(0, predicted_amihud)
        except Exception as e:
            print(f"Prediction error for {ticker}: {e}")
            return current_metrics['rolling_vol'], current_metrics['amihud']

    def compute_liquidity_score(self, vol_metric, amihud_metric, zero_vol_ratio, 
                              vol_threshold=1000, amihud_threshold=0.0001, zero_ratio_threshold=0.15):
        """Compute composite liquidity score (0-1, higher = more liquid)"""
        vol_score = 1 / (1 + np.exp(-(vol_metric - vol_threshold) / (vol_threshold * 0.5)))
        amihud_score = 1 / (1 + np.exp((amihud_metric - amihud_threshold) / (amihud_threshold * 2)))
        zero_vol_score = 1 - min(1, zero_vol_ratio / zero_ratio_threshold)
        
        composite_score = 0.5 * vol_score + 0.3 * amihud_score + 0.2 * zero_vol_score
        
        return {
            'composite_score': composite_score,
            'vol_score': vol_score,
            'amihud_score': amihud_score,
            'zero_vol_score': zero_vol_score,
            'is_liquid': composite_score > 0.5
        }

    def predict_adaptive_liquidity(self, trade_date, current_time, tickers=None, 
                                 vol_threshold=1000, zero_ratio_threshold=0.15, 
                                 amihud_threshold=0.0001):
        """Main method: Predict liquidity adaptively based on current time"""
        trade_datetime = pd.Timestamp(f"{trade_date} {current_time}")
        
        if trade_datetime.hour == 9 and trade_datetime.minute <= 15:
            end_data_time = trade_datetime.replace(hour=9, minute=15) - timedelta(days=1)
            start_data_time = end_data_time - timedelta(days=self.lookback_days)
            use_current_day = False
        else:
            start_data_time = trade_datetime.replace(hour=9, minute=15)
            end_data_time = trade_datetime
            use_current_day = True
        
        if tickers is None:
            tickers = self.reader.get_available_tickers(trade_date)
        
        results = {}
        
        for ticker in tickers:
            try:
                historical_df = self.reader.get_stock_data(
                    [ticker],
                    start=start_data_time - timedelta(days=self.lookback_days),
                    end=start_data_time,
                    columns=["date", "time", "close", "volume", "open"],
                    impute=False
                ).get(ticker, pd.DataFrame())
                
                current_day_df = None
                if use_current_day:
                    current_day_df = self.reader.get_stock_data(
                        [ticker],
                        start=start_data_time,
                        end=end_data_time,
                        columns=["date", "time", "close", "volume", "open"],
                        impute=False
                    ).get(ticker, pd.DataFrame())
                
                combined_df = historical_df
                if current_day_df is not None and not current_day_df.empty:
                    combined_df = pd.concat([historical_df, current_day_df]).sort_index()
                
                if combined_df.empty:
                    print(f"No data available for {ticker}")
                    continue
                
                current_metrics = self.compute_base_liquidity_metrics(combined_df, ticker)
                if current_metrics is None:
                    print(f"Failed to compute metrics for {ticker}")
                    continue
                
                self.update_ema_state(ticker, current_metrics, trade_datetime)
                
                predicted_vol, predicted_amihud = self.predict_future_liquidity(
                    ticker, trade_datetime, current_metrics
                )
                
                current_score = self.compute_liquidity_score(
                    current_metrics['rolling_vol'], 
                    current_metrics['amihud'],
                    current_metrics['zero_vol_ratio'],
                    vol_threshold, amihud_threshold, zero_ratio_threshold
                )
                
                predicted_score = self.compute_liquidity_score(
                    predicted_vol, predicted_amihud,
                    current_metrics['zero_vol_ratio'],
                    vol_threshold, amihud_threshold, zero_ratio_threshold
                )
                
                results[ticker] = {
                    'timestamp': trade_datetime,
                    'current_liquidity': current_score,
                    'predicted_liquidity': predicted_score,
                    'current_metrics': current_metrics,
                    'predicted_metrics': {
                        'volume': predicted_vol,
                        'amihud': predicted_amihud
                    },
                    'ema_state': self.liquidity_states[ticker].copy() if ticker in self.liquidity_states else None
                }
                
            except Exception as e:
                print(f"Error processing {ticker}: {e}")
                continue
        
        return results

    def save_models(self):
        """Save trained models and scaler"""
        model_data = {
            'volume_predictor': self.volume_predictor,
            'amihud_predictor': self.amihud_predictor,
            'feature_scaler': self.feature_scaler,
            'ema_params': self.ema_params
        }
        
        with open(self.model_path, 'wb') as f:
            pickle.dump(model_data, f)

    def load_models(self):
        """Load saved models"""
        if self.model_path.exists():
            try:
                with open(self.model_path, 'rb') as f:
                    model_data = pickle.load(f)
                
                self.volume_predictor = model_data.get('volume_predictor')
                self.amihud_predictor = model_data.get('amihud_predictor')
                self.feature_scaler = model_data.get('feature_scaler', RobustScaler())
                self.ema_params = model_data.get('ema_params', self.ema_params)
                
                print("Models loaded successfully")
            except Exception as e:
                print(f"Error loading models: {e}")

    def save_normalization_params(self):
        """Save normalization parameters"""
        with open(self.normalization_path, 'wb') as f:
            pickle.dump(self.normalization_params, f)

    def load_normalization_params(self):
        """Load normalization parameters"""
        if self.normalization_path.exists():
            try:
                with open(self.normalization_path, 'rb') as f:
                    self.normalization_params = pickle.load(f)
                print("Normalization parameters loaded successfully")
            except Exception as e:
                print(f"Error loading normalization parameters: {e}")

# reader = StockDataReader()
liquid_tickers = liquid_data.keys()
predictor = AdaptiveLiquidityPredictor(reader)

results_open = predictor.predict_adaptive_liquidity(
    trade_date="2022-02-01",
    current_time="09:15",
    tickers=liquid_tickers
)

results_mid = predictor.predict_adaptive_liquidity(
    trade_date="2022-02-01", 
    current_time="14:30",
    tickers=liquid_tickers
)

for ticker, result in results_open.items():
    current_liq = result['current_liquidity']
    predicted_liq = result['predicted_liquidity']
    
    print(f"\n{ticker} at 09:15:")
    print(f"Current Liquidity Score: {current_liq['composite_score']:.3f} ({'Liquid' if current_liq['is_liquid'] else 'Illiquid'})")
    print(f"Predicted Liquidity Score: {predicted_liq['composite_score']:.3f} ({'Liquid' if predicted_liq['is_liquid'] else 'Illiquid'})")

In [ ]:

for ticker, result in results_mid.items():
    current_liq = result['current_liquidity']
    predicted_liq = result['predicted_liquidity'] 
    
    print(f"\n{ticker} at 09:15:")
    print(f"Current Liquidity Score: {current_liq['composite_score']:.3f} ({'Liquid' if current_liq['is_liquid'] else 'Illiquid'})")
    print(f"Predicted Liquidity Score: {predicted_liq['composite_score']:.3f} ({'Liquid' if predicted_liq['is_liquid'] else 'Illiquid'})")

In [ ]:
break

In [ ]:
zero_volume_tickers = {ticker  for ticker, df in dfs.items() if df.volume.sum() == 0}
len(dfs)

In [ ]:
dfs = {ticker: df for ticker, df in dfs.items() if ticker not in ticker zero_volume_tickers}


In [ ]:
# dfs = get_stock_data(
#     tickers,
#     impute=False
# )

Mean trading volume for all the stocks for the month

In [ ]:
overall_mean_monthly_traded_volume = sum(df.volume.mean() for _, df in dfs.items())/len(dfs)

 Here we find the stocks whichh have high trading mean trading volumes for atleast half the month

In [ ]:
import matplotlib.pyplot as plt

# Calculate high volume day count for each stock
high_volume_day_counts = {
    s: (df.groupby(df.index.date)['volume'].mean() > overall_mean_monthly_traded_volume).sum().item()
    for s, df in dfs.items()
}

# Histogram 
plt.figure(figsize=(10, 6))
plt.hist(high_volume_day_counts.values(), bins=10, edgecolor='black')
plt.xlabel('Number of High-Volume Days')
plt.ylabel('Number of Stocks')
plt.title('Distribution of High-Volume Days Across Stocks')
plt.grid(True)
plt.show()


We observe that for more than half of January, there was sufficient liquidity in the market for some of the stocks. A huge number of stocks have been illiquid 

In [ ]:
liquid_stocks = [ s for s, v in high_volume_day_counts.items() if v > 15]

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

def calculate_half_life(price_series):
    """
    Calculates the half-life of mean reversion for a price series using linear regression.
    
    Reference:
        Chan, E. P. (2013). Algorithmic Trading: Winning Strategies and Their Rationale. 
        John Wiley & Sons.
    """
    delta_p = price_series.diff().dropna()
    p_lag = price_series.shift(1).dropna()
    
    # Align indices explicitly - keep only indices that appear in both
    common_idx = delta_p.index.intersection(p_lag.index)
    delta_p = delta_p.loc[common_idx]
    p_lag = p_lag.loc[common_idx]
    
    model = OLS(delta_p.values, add_constant(p_lag.values))
    res = model.fit()
    beta = res.params[1]
    halflife = -np.log(2) / beta if beta < 0 else np.inf
    return halflife

def hurst_exponent(ts):
    lags = range(2, 100)
    tau = [np.sqrt(np.std(np.subtract(ts[lag:], ts[:-lag]))) for lag in lags]
    poly = np.polyfit(np.log(lags), np.log(tau), 1)
    return poly[0] * 2.0

def mean_reversion_vector(price_series):
    price_series = price_series.dropna()
    
    adf_stat = adfuller(price_series)[0]
    half_life = calculate_half_life(price_series)
    hurst = hurst_exponent(price_series.values)
    
    returns = price_series.pct_change().dropna()
    vol = returns.std()
    
    return np.array([adf_stat, half_life, hurst, vol])

# Now your embedding extraction code can stay the same:
embedding_list = []
valid_stocks = []

# for ticker in liquid_stocks:
#     df = dfs[ticker]
#     price_series = df['close']
    
#     try:
#         vec = mean_reversion_vector(price_series)
#         if np.all(np.isfinite(vec)):
#             embedding_list.append(vec)
#             valid_stocks.append(ticker)
#     except Exception as e:
#         print(f"Failed for {ticker}: {e}")

# embeddings = np.vstack(embedding_list)


In [ ]:
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler

def process_ticker(ticker, price_series):
    """Compute mean-reversion vector for one ticker"""
    try:
        vec = mean_reversion_vector(price_series)
        if np.all(np.isfinite(vec)):
            return ticker, vec
    except Exception as e:
        print(f"Failed for {ticker}: {e}")
    return None

# Prepare inputs first (avoid passing big dfs dict to workers)
inputs = [(ticker, dfs[ticker]['close']) for ticker in liquid_stocks]

results = Parallel(n_jobs=8, backend="loky")(  # try 8 workers first
    delayed(process_ticker)(ticker, price_series) 
    for ticker, price_series in inputs
)

valid_results = [r for r in results if r is not None]
valid_stocks, embedding_list = zip(*valid_results)
embeddings = np.vstack(embedding_list)

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# Cosine similarity matrix
cos_sim = cosine_similarity(embeddings)

# Cluster 
cluster_model = AgglomerativeClustering(n_clusters=20, linkage='average')

distances = 1 - cos_sim

labels = cluster_model.fit_predict(distances)

# Now labels[i] gives cluster of valid_stocks[i]


In [ ]:
! pip install umap-learn


In [ ]:

import umap.umap_ as umap 

reducer = umap.plot.interactive(min_dist =0.002, n_components=6, n_neighbors=30)
embeddings_umap = reducer.fit_transform(embeddings)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(embeddings_umap[:, 0], embeddings_umap[:, 1], c=labels, cmap='tab10', s=50, alpha=0.8)
plt.title("Stocks clustered by mean-reversion indicators (UMAP 2D projection)")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.colorbar(scatter, label='Cluster label')
plt.grid(True)
plt.show()


In [ ]:
from collections import defaultdict

clustered_tickers = defaultdict(list)
for ticker, label in zip(tickers, labels):
    clustered_tickers[label].append(ticker)

for label in sorted(clustered_tickers):
    print(f"\nCluster {label} ({len(clustered_tickers[label])} tickers):")
    print(", ".join(clustered_tickers[label]))


In [ ]:
from itertools import combinations
from statsmodels.tsa.stattools import coint
import pandas as pd

def evaluate_pair(label, ticker1, ticker2):
    try:
        s1 = dfs[ticker1]['close']
        s2 = dfs[ticker2]['close']
        joined = pd.concat([s1, s2], axis=1, join='inner').dropna()
        if len(joined) < 50:
            return None
        s1_aligned = joined.iloc[:, 0]
        s2_aligned = joined.iloc[:, 1]
        score, pvalue, _ = coint(s1_aligned, s2_aligned)
        return {
            'cluster': label,
            'pair': (ticker1, ticker2),
            'cointegration_pvalue': pvalue
        }
    except Exception as e:
        return None

all_pairs = []
for label, ticker_list in clustered_tickers.items():
    if 2 <= len(ticker_list) <= 10:
        for ticker1, ticker2 in combinations(ticker_list, 2):
            all_pairs.append((label, ticker1, ticker2))
pair_results = []
for label, ticker1, ticker2 in all_pairs:
    result = evaluate_pair(label, ticker1, ticker2)
    if result is not None:
        pair_results.append(result)


In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.stattools import adfuller

for result in pair_results:
    t1, t2 = result['pair']
    s1 = dfs[t1]['close']
    s2 = dfs[t2]['close']
    joined = pd.concat([s1, s2], axis=1, join='inner').dropna()
    y = joined.iloc[:,0].values
    x = joined.iloc[:,1].values
    beta = OLS(y, add_constant(x)).fit().params[1]
    spread = y - beta * x
    adf_stat, adf_pvalue, *_ = adfuller(spread)
    result['spread_adf_stat'] = adf_stat
    result['spread_adf_pvalue'] = adf_pvalue
    result['spread_std'] = np.std(spread)


In [ ]:
good_pairs = [
    r for r in pair_results
    if r['cointegration_pvalue'] < 0.05 and r['spread_adf_pvalue'] < 0.05
]
#  sort by (1) p-value, (2) spread_std, (3) cluster size
good_pairs = sorted(good_pairs, key=lambda r: (r['cointegration_pvalue'], -r['spread_std']))


In [ ]:
import pandas as pd
df_report = pd.DataFrame(good_pairs)
print(df_report[['cluster', 'pair', 'cointegration_pvalue', 'spread_adf_pvalue', 'spread_std']])


In [ ]:
pair = good_pairs[0]['pair']
s1 = dfs[pair[0]]['close']
s2 = dfs[pair[1]]['close']
joined = pd.concat([s1, s2], axis=1, join='inner').dropna()
y = joined.iloc[:,0].values
x = joined.iloc[:,1].values
beta = OLS(y, add_constant(x)).fit().params[1]
spread = y - beta * x

plt.figure(figsize=(12,5))
plt.plot(joined.index, spread)
plt.title(f"Spread between {pair[0]} and {pair[1]}")
plt.xlabel("Time")
plt.ylabel("Spread")
plt.show()
